In [5]:
from pmdarima import auto_arima
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [6]:
# 創建時間窗口的訓練集數據
def create_dataset(closing_prices, time_window):
    features,targets = [],[]
    for i in range(len(closing_prices) - time_window):
        features.append(closing_prices.iloc[i:i+time_window])
        targets.append(closing_prices.iloc[i+time_window])   
    return np.array(features),np.array(targets)

In [7]:
# load dataset
# file_list=['BTCUSD-all','BTCUSD-N2Y','BTCUSD-N4Y',
#           'ETHUSD-all','ETHUSD-N2Y','ETHUSD-N4Y',           
#            'USDTUSD-all','USDTUSD-N2Y','USDTUSD-N4Y',  
#            'BNBUSD-all','BNBUSD-N2Y','BNBUSD-N4Y']

# file_list=['BTCUSD-1m1h','ETHUSD-1m1h','USDTUSD-1m1h','BNBUSD-1m1h']
file_list=['BTCUSD-N4Y']
spilt_list=[0.3]
look_backs=[7]


files_list=list()
spilts_list=list()
Classifiers_list=list()
lb_list=list()

train_mae=list()
train_rmse=list()
test_mae=list()
test_rmse=list()


In [8]:
for file in file_list:
    for splits in spilt_list:
        for lb in look_backs:
            files_list.append(file)
            spilts_list.append(splits)
            lb_list.append(lb)

            data = pd.read_csv(f'../Data/{file}.csv') 
            train_data, test_data = train_test_split(data['Close'], test_size=splits, shuffle=False)

        
            trainX, trainY = create_dataset(train_data, lb)
            testX, testY = create_dataset(test_data, lb)
            tscv = TimeSeriesSplit(n_splits=5)
            best_model = None
            best_rmse = np.inf

   
            best_model = auto_arima(trainY,exogenous=trainX,max_p=20,max_q=20,seasonal=False)
            print(f'資料集{file} {splits} {lb}')
            best_model_params = best_model.get_params()['order']
            print("最佳模型參數:ARIMA", best_model_params)
            Classifiers_list.append(f'ARIMA{best_model_params}')
            
            # forecast
            forecast = best_model.predict(n_periods=len(testY))
                

            test_data_mae=mean_absolute_error(testY,forecast)
            test_data_rmse=mean_squared_error(testY,forecast)**0.5        
            test_mae.append(test_data_mae)
            test_rmse.append(test_data_rmse)       

            # predict train
            train_forecast = best_model.predict_in_sample()    

            train_data_mae=mean_absolute_error(trainY,train_forecast)
            train_data_rmse=mean_squared_error(trainY,train_forecast)**0.5
            train_mae.append(train_data_mae)
            train_rmse.append(train_data_rmse) 

            print(f'MAE train:{train_data_mae:.4f} test:{test_data_mae:.4f}')   
            print(f'RMSE train:{train_data_rmse:.4f} test:{test_data_rmse:.4f}')         
        
        

資料集BTCUSD-N4Y 0.3 7
最佳模型參數:ARIMA (2, 1, 2)
MAE train:622.4628 test:9572.1433
RMSE train:1127.7645 test:10703.5583


In [10]:
Summary={'Data':files_list,'Classifier':Classifiers_list,'Test Split':spilts_list,'look back':lb_list,
            'train_mae':train_mae,          
            'train_rmse':train_rmse,          
            'test_mae':test_mae,           
            'test_rmse':test_rmse            
         }

df_Summary = pd.DataFrame(Summary)
df_Summary.to_excel("Summary-AutoARIMA(12lb).xlsx",index=False) 
df_Summary

,Data,Classifier,Test Split,look back,train_mae,train_rmse,test_mae,test_rmse
0,BTCUSD-1m1h,"ARIMA(0, 1, 0)",0.3,12,179.536760,1577.956168,1722.054898,1761.671949
1,ETHUSD-1m1h,"ARIMA(0, 1, 0)",0.3,12,11.173176,82.620259,118.097727,120.778397
2,USDTUSD-1m1h,"ARIMA(1, 1, 1)",0.3,12,0.002132,0.045095,0.000196,0.000231
3,BNBUSD-1m1h,"ARIMA(0, 1, 0)",0.3,12,1.544990,10.352066,4.639389,5.556377
